In [ ]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config


In [ ]:
# set up login
Cpassword = nvda_config.mysql_password
api_k = nvda_config.twelve_api_nvda
base_url = 'https://api.twelvedata.com'


In [ ]:
# Estab;ish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()




In [ ]:

# Define the start and end times
start_time = time(6, 30)
end_time = time(13, 0)

while True:
    current_time = datetime.now().time()
    
    if start_time <= current_time <= end_time:
       # Fetch time series data and indicators
    time_series = td.time_series(
        symbol=symbol,
        interval=interval,
        outputsize=outputsize,
        start_date=current_date.strftime("%Y-%m-%d"),
        end_date=chunk_end_date.strftime("%Y-%m-%d"),
    ).with_macd().with_ema().with_rsi()
    
    # Convert to pandas Dataframes
    time_series_df = time_series.as_pandas()
    time_series_df = time_series_df.astype(str)

    # Prepare for database insertion
    time_series_data = []
    indicator_data = []

    # Process and align data
    for index, row in time_series_df.iterrows():
        # Extract the datetime string (already in correct format)
        datetime_str = row.name.strftime("%Y-%m-%d %H:%M:%S")

        time_series_data.append(
            (
                symbol,
                datetime_str,
                row["open"],
                row["high"],
                row["low"],
                row["close"],
                row["volume"],
            )
        )
        if pd.notna(row["macd"]):  # Check if MACD is not NaN
            indicator_data.append(
                (
                    symbol,
                    datetime_str,
                    row["macd"],
                    row["macd_signal"],
                    row["macd_hist"],
                    row["ema"],
                    row["rsi"],
                )
            )


    # Insert tick data into MySQL
    sql_time_series = """
    INSERT INTO nvda_new (symbol, datetime, open, high, low, close, volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    mycursor.executemany(sql_time_series, time_series_data)

    # Insert indicator data into MySQL
    sql_indicators = """
    INSERT INTO nvda_TI_new (symbol, bydatetime, macd, bysignal, byhistogram, ema, rsi)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    mycursor.executemany(sql_indicators, indicator_data)
else:
        # Sleep for a short period before checking the time again
    time.sleep(10)
    print("markets closed")

# ... existing code ...